In [4]:
import numpy as np 
import pandas as pd
import os
import gc
import re

# feature libraries
from scipy.sparse import hstack, csr_matrix
from sklearn import preprocessing

In [8]:
data_dir = '../input/kaggle_data'
feat_dir = '../input/features'

In [11]:
used_cols = ['item_id','image']

train = pd.read_csv(f'{data_dir}/train.csv', index_col="item_id", usecols=used_cols)
test = pd.read_csv(f'{data_dir}/test.csv', index_col="item_id", usecols=used_cols)

df = pd.concat([train, test], axis=0)

train_idx = train.index
test_idx = test.index

train_img_nima = pd.read_csv(f'{feat_dir}/train_img_nima.csv')
test_img_nima = pd.read_csv(f'{feat_dir}/test_img_nima.csv')

df_img_nima = pd.concat([train_img_nima,test_img_nima],axis=0)

del train, test
gc.collect()

14

In [12]:
df = df.reset_index().merge(df_img_nima, how="left", on='image').set_index('item_id')


In [14]:
nima_cols = ["mobile_mean", "mobile_std","inception_mean", "inception_std", "nasnet_mean", "nasnet_std"]
df[nima_cols] = df[nima_cols].fillna(0)
for c in nima_cols:
    df[c] = (df[c] - np.mean(df[c]))/np.std(df[c])

In [18]:
df.drop(['image'], axis=1, inplace=True)

In [19]:
df.columns[df.isna().any()].tolist()

[]

In [21]:
feat_cols = list(df.columns)
feat_cols

['mobile_mean',
 'mobile_std',
 'inception_mean',
 'inception_std',
 'nasnet_mean',
 'nasnet_std']

In [26]:
train_nima = df.loc[train_idx,:]
train_nima.head()

,mobile_mean,mobile_std,inception_mean,inception_std,nasnet_mean,nasnet_std
item_id,,,,,,
b912c3c6a6ad,0.376462,0.196650,0.038229,0.360435,-0.071322,0.059919
2dac0150717d,0.380049,0.214786,0.588905,0.167156,0.240389,-0.033727
ba83aefab5dc,0.542563,0.095300,0.846048,0.241327,0.945804,0.401474
02996f1dd2ea,0.440850,0.415528,0.831658,0.253381,0.486166,0.321294
7c90be56d2ab,0.051788,0.310443,0.310625,0.161061,0.436753,0.278871


In [23]:
test_nima = df.loc[test_idx,:]
test_nima.head()

,mobile_mean,mobile_std,inception_mean,inception_std,nasnet_mean,nasnet_std
item_id,,,,,,
6544e41a8817,0.141726,0.081817,0.354108,0.287142,0.342020,0.375762
65b9484d670f,-3.343175,-3.347877,-3.209252,-3.362997,-3.191177,-3.358589
8bab230b2ecd,0.530402,0.453288,0.344191,0.330563,0.398008,0.501167
8e348601fefc,-3.343175,-3.347877,-3.209252,-3.362997,-3.191177,-3.358589
8bd2fe400b89,0.197106,0.255780,-0.233574,0.095748,-0.191619,0.192074


In [27]:
train_nima.to_csv(f'{feat_dir}/train_nima.csv', index=True, header=True)
test_nima.to_csv(f'{feat_dir}/test_nima.csv', index=True, header=True)